In [19]:
import pandas as pd
import numpy as np
import re
import nltk
import json

In [20]:
master_list = pd.read_csv("..//data/searchResults.csv")

## Remove extra rows


In [21]:
def find_bad_row(row, col, term):
    if re.search(term, row[col], flags = re.I) is not None:
        return True
    return False

In [22]:
bad_indexes = master_list.apply(lambda row: find_bad_row(row, "License Number", "license number"), axis = 1)
master_list = master_list[~bad_indexes].reset_index(drop = True)

## Add extra fields

In [23]:
def break_fields(row, col, string):
    if pd.isnull(row[col]):
        return ""
    
    test = re.search(string, row[col], flags = re.I)
    if test != None:
        return test.group()
    
    return ""

def get_city(row, col):
    street_abbrev = [" w ", " blvd ", " st ", " rd ", " pkwy ", " ave ", 
                     " ctr ", " cir ", " ct ", " dr ", " ln ", " lk ", 
                     " lp ", " pl ", " sq ", " tr ", " e ", " n ", " s ", 
                     " hwy ", " way ", " wy ", ]
    fix = {"[0-9]": "", "\s+": " ", "san fran": "san francisco", "avenue": "", "highway": ""}
    if pd.isnull(row[col]):
        return ""
    
    left_limit = 0
    check_list = []
    for i in street_abbrev:
        street = re.search(i, row[col], flags = re.I)
        if street != None:  
            check_list.append(street.end())
        else:
            check_list.append(0)
    left_limit = max(check_list)
    
    right_limit = re.search(", ", row[col], flags = re.I)
    if right_limit != None:
        right_limit = right_limit.start()
        
    tmp = row[col][left_limit: right_limit].strip()
    
    for i in fix:
        tmp = re.sub(i, fix[i], tmp, flags = re.I)
        
    return tmp.strip()

def get_city(row, col):
    street_abbrev = [" w ", " blvd ", " st ", " rd ", " pkwy ", " ave ", 
                     " ctr ", " cir ", " ct ", " dr ", " ln ", " lk ", 
                     " lp ", " pl ", " sq ", " tr ", " e ", " n ", " s ", 
                     " hwy ", " way ", " wy ", ]
    fix = {"[0-9]": "", "\s+": " ", "san fran": "san francisco", "avenue": "", "highway": "", "san franciscocisco": "san francisco"}
    if pd.isnull(row[col]):
        return ""
    
    left_limit = 0
    check_list = []
    for i in street_abbrev:
        street = re.search(i, row[col], flags = re.I)
        if street != None:  
            check_list.append(street.end())
        else:
            check_list.append(0)
    left_limit = max(check_list)
    
    right_limit = re.search(", ", row[col], flags = re.I)
    if right_limit != None:
        right_limit = right_limit.start()
        
    tmp = row[col][left_limit: right_limit].strip()
    
    for i in fix:
        tmp = re.sub(i, fix[i], tmp, flags = re.I)
        
    return tmp.strip()

def get_email(row, col):
    replace = {"email": "", "Email": "", "-": ""}
    if pd.isnull(row[col]):
        return ""
    check = re.search("email.{,1}?-.+?@.+?\s", row[col], flags = re.I)
    
    if check is not None:
        check = check.group()
        for i in replace:
            check = check.replace(i, replace[i])
        return check.strip()
    
    return ""

def get_website(row, col):
    if pd.isnull(row[col]):
        return ""
    check = re.search("website-", row[col], flags = re.I)
    
    if check is not None:
        check = row[col][check.end():]
        return check.strip()
    
    return ""

def get_name(row, col):
    if pd.isnull(row[col]):
        return ""
    check = re.search(":", row[col], flags = re.I)
    
    if check is not None:
        check = row[col][:check.end() - 1]
        return check.strip()
    
    return ""

def get_phone(row, col):
    if pd.isnull(row[col]):
        return ""
    
    check = re.search("phone-.+?\s", row[col], flags = re.I)
    #print(check)
    
    if check is not None:
        check = re.sub("-|:|\(|\)|phone", "", check.group(), flags = re.I)
        return check.strip()
        
    return ""

In [24]:
master_list["zip_code"] = master_list.apply(lambda row: break_fields(row, "Premise Address", "([0-9]{5})|([0-9]{9})"), axis = 1)
master_list["city"] = master_list.apply(lambda row: get_city(row, "Premise Address"), axis = 1).str.lower()
master_list["email"] = master_list.apply(lambda row: get_email(row, "Business Contact Information"), axis = 1).str.lower()
master_list["website"] = master_list.apply(lambda row: get_website(row, "Business Contact Information"), axis = 1).str.lower()
master_list["company_name"] = master_list.apply(lambda row: get_name(row, "Business Contact Information"), axis = 1).str.lower()
master_list["phone"] = master_list.apply(lambda row: get_phone(row, "Business Contact Information"), axis = 1).astype(str)
master_list["License_no_dash"] = master_list["License Number"].str.replace("-", "")
#master_list["master_index"] = master_list.index

In [30]:
retailers

,License Number,License Type,Business Owner,Business Contact Information,Business Structure,Premise Address,Status,Issue Date,Expiration Date,Activities,Adult-Use/Medicinal,zip_code,city,email,website,company_name,phone,License_no_dash
1,C10-18-0000287-TEMP,Cannabis - Retailer Temporary License,Angel Fernandez,MOVOCAN : Email- medigroupangel@gmail.com : Ph...,Corporation,"1632 GATEWAY RD CALEXICO, CA 922319530 County:...",Active,07/29/2019,12/31/2018,N/A for this license type,Medicinal,92231,calexico,medigroupangel@gmail.com,www.movocan.com,movocan,7605409358,C10180000287TEMP
2,C9-18-0000229-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Angel Fernandez,MOVOCAN : Email- medigroupangel@gmail.com : Ph...,Corporation,"CALEXICO, CA 922319530 County: IMPERIAL",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,92231,calexico,medigroupangel@gmail.com,www.movocan.com,movocan,7605409358,C9180000229TEMP
4,C9-18-0000230-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,martha Medina,RGG LLC : Ruby's Green Garden : Email- rubysg...,Limited Liability Company,"OAKLAND, CA 946012132 County: ALAMEDA",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,94601,oakland,rubysgreengarden.inc@gmail.com,,rgg llc,9259151028,C9180000230TEMP
7,C10-18-0000288-TEMP,Cannabis - Retailer Temporary License,Basil Bailey,ORIGINAL BALBOA CAREGIVERS : Email- balboacare...,Corporation,"9419 MASON AVE CHATSWORTH, CA 913115204 County...",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,91311,chatsworth,balboacaregivers@yahoo.com,,original balboa caregivers,7477776255,C10180000288TEMP
11,C9-18-0000231-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Rickey McCullough,MCCULLOUGHS MANAGEMENT GROUP LLC : MCCULLOUGH...,Limited Liability Company,"OAKLAND, CA 946014902 County: ALAMEDA",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,94601,oakland,rickeymcc444@gmail.com,,mcculloughs management group llc,5107984681,C9180000231TEMP
12,C9-18-0000232-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Gregory Reeves,Lit Delivery Service : Lit Delivery Service :...,Limited Liability Company,"OAKLAND, CA 94612 County: ALAMEDA",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,94612,oakland,litoakland@gmail.com,www.litoakland.com,lit delivery service,5108669438,C9180000232TEMP
13,C9-18-0000233-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Sheila Cross,THE FARMER'S FLOWER : Email- thefarmersflower@...,Corporation,"OAKLAND, CA 946213030 County: ALAMEDA",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,94621,oakland,thefarmersflower@gmail.com,,the farmer's flower,9255198309,C9180000233TEMP
18,C9-18-0000234-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Carson McCarty,JAHnetics Delivery : Jahnetics Delivery : Ema...,Corporation,"SAN FRANCISCO, CA 941114401 County: SAN FRANCI...",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,94111,san francisco,carsonjmccarty@gmail.com,www.jahnetics.com,jahnetics delivery,4155963315,C9180000234TEMP
21,C9-18-0000235-TEMP,Cannabis - Retailer Nonstorefront Temporary Li...,Amardeep Bhullar,WEED ON WHEELS LLC : Weed On Wheels LLC : Ema...,Limited Liability Company,"OAKLAND, CA 946212102 County: ALAMEDA",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,94621,oakland,amarbhullar@gmail.com,,weed on wheels llc,5106100555,C9180000235TEMP
23,C10-18-0000289-TEMP,Cannabis - Retailer Temporary License,Michael O'Leary,"Medallion Wellness, Inc. : Email- michael@meda...",Corporation,"341 Business PKWY Atwater, CA 95301 County:",Active,07/29/2019,12/31/2018,N/A for this license type,BOTH,95301,atwater,michael@medallionwellness.com,,"medallion wellness, inc.",2094024995,C10180000289TEMP


In [25]:
master_list.to_csv("..//data/searchResultsClean.csv", index = False)

In [27]:
retailers = master_list[master_list["License Type"].isin(["Cannabis - Retailer Temporary License", "Cannabis - Retailer Nonstorefront Temporary License"])]

In [29]:
retailers.to_csv("..//data/searchResultsRetailers.csv", index = False)